In [1]:
import os
import sys

BOOTPATH = os.path.dirname(sys.path[0])
sys.path.insert(0, BOOTPATH)
import dpeeg
print(dpeeg.__version__)

0.3.5


In [2]:
from dpeeg.data import transforms, datasets

# trans = transforms.ComposeTransforms(
#     transforms.Normalization(),
#     transforms.Unsqueeze(),
#     transforms.Augmentation('segmentation_and_reconstruction', multiply=0, n=8),
# )

data = datasets.BCICIV2B(test_sessions=[3, 4], verbose='WARNING')

Tensorflow not install, you could not use those pipelines
To use the get_shape_from_baseconcar, InputShapeSetterEEG, BraindecodeDatasetLoaderyou need to install `braindecode`.`pip install braindecode` or Please refer to `https://braindecode.org`.
sub1-session_0-run_0: 120
sub1-session_1-run_0: 120
sub1-session_2-run_0: 160
sub1-session_3-run_0: 160
sub1-session_4-run_0: 160
sub2-session_0-run_0: 120
sub2-session_1-run_0: 120
sub2-session_2-run_0: 160
sub2-session_3-run_0: 120
sub2-session_4-run_0: 160
sub3-session_0-run_0: 120
sub3-session_1-run_0: 120
sub3-session_2-run_0: 160
sub3-session_3-run_0: 160
sub3-session_4-run_0: 160
sub4-session_0-run_0: 120
sub4-session_1-run_0: 140
sub4-session_2-run_0: 160
sub4-session_3-run_0: 160
sub4-session_4-run_0: 160
sub5-session_0-run_0: 120
sub5-session_1-run_0: 140
sub5-session_2-run_0: 160
sub5-session_3-run_0: 160
sub5-session_4-run_0: 160
sub6-session_0-run_0: 120
sub6-session_1-run_0: 120
sub6-session_2-run_0: 160
sub6-session_3-run_0: 160

In [3]:
for sub, d in data.items():
    print(f'sub{sub}: train={d["train"][0].shape},test={d["test"][0].shape}')

sub1: train=(400, 3, 1000),test=(320, 3, 1000)
sub2: train=(400, 3, 1000),test=(280, 3, 1000)
sub3: train=(400, 3, 1000),test=(320, 3, 1000)
sub4: train=(420, 3, 1000),test=(320, 3, 1000)
sub5: train=(420, 3, 1000),test=(320, 3, 1000)
sub6: train=(400, 3, 1000),test=(320, 3, 1000)
sub7: train=(400, 3, 1000),test=(320, 3, 1000)
sub8: train=(440, 3, 1000),test=(320, 3, 1000)
sub9: train=(400, 3, 1000),test=(320, 3, 1000)


In [1]:
from moabb.datasets import BNCI2014004

dataset = BNCI2014004()
data = dataset.get_data()

Tensorflow not install, you could not use those pipelines
To use the get_shape_from_baseconcar, InputShapeSetterEEG, BraindecodeDatasetLoaderyou need to install `braindecode`.`pip install braindecode` or Please refer to `https://braindecode.org`.


In [2]:
import mne

raw = data[1]['session_3']['run_0']
events = mne.find_events(raw)
events

160 events found
Event IDs: [1 2]


array([[ 54349,      0,      2],
       [ 57070,      0,      1],
       [ 59327,      0,      2],
       [ 61596,      0,      1],
       [ 63848,      0,      1],
       [ 66120,      0,      1],
       [ 68563,      0,      2],
       [ 70880,      0,      1],
       [ 73282,      0,      1],
       [ 75596,      0,      1],
       [ 78221,      0,      2],
       [ 80542,      0,      1],
       [ 83164,      0,      2],
       [ 85763,      0,      1],
       [ 88064,      0,      2],
       [ 90427,      0,      1],
       [ 93018,      0,      1],
       [ 95479,      0,      1],
       [ 97947,      0,      1],
       [100252,      0,      1],
       [102868,      0,      2],
       [105510,      0,      2],
       [108035,      0,      2],
       [110744,      0,      1],
       [113255,      0,      2],
       [115536,      0,      2],
       [117838,      0,      2],
       [120554,      0,      2],
       [122968,      0,      1],
       [125532,      0,      2],
       [12